In [10]:
import pandas as pd
import numpy as np
import scipy.stats as stats

class OLS:
    def __init__(self, y, X):
        self.y = y
        self.X_data = X  # Original X for name extraction

        # Handle feature names
        if isinstance(X, (pd.DataFrame, pd.Series)):
            self.X_names = ['const'] + (X.columns.tolist() if isinstance(X, pd.DataFrame) else [X.name])
        else:
            num_features = X.shape[1] if len(X.shape) > 1 else 1
            self.X_names = ['const'] + [f'x{i}' for i in range(1, num_features + 1)]

        # Add constant to X matrix
        self.X = np.column_stack((np.ones(len(X)), X.values if isinstance(X, (pd.DataFrame, pd.Series)) else X))
        self.nobs = len(y)

    def fit(self):
        # Coefficients
        XtX = np.dot(self.X.T, self.X)
        XtX_inv = np.linalg.inv(XtX)
        self.coef = np.dot(XtX_inv, np.dot(self.X.T, self.y))

        # Residuals and variance
        self.resid = self.y - np.dot(self.X, self.coef)
        self.ssr = np.sum(self.resid**2)
        self.sigma2 = self.ssr / (self.nobs - self.X.shape[1])

        # Standard errors
        self.se = np.sqrt(np.diag(XtX_inv * self.sigma2))

        # T-stats and p-values
        self.t_values = self.coef / self.se
        self.p_values = 2 * (1 - stats.t.cdf(np.abs(self.t_values), self.nobs - self.X.shape[1]))

        # R-squared
        self.sst = np.sum((self.y - np.mean(self.y))**2)
        self.r_squared = 1 - self.ssr / self.sst
        self.adj_r_squared = 1 - (self.ssr/(self.nobs-self.X.shape[1])) / (self.sst/(self.nobs-1))

        # F-statistic
        self.f_stat = (self.r_squared / (self.X.shape[1]-1)) / ((1 - self.r_squared) / (self.nobs - self.X.shape[1]))
        self.f_pvalue = 1 - stats.f.cdf(self.f_stat, self.X.shape[1]-1, self.nobs - self.X.shape[1])

        # Log-Likelihood
        sigma2_mle = self.ssr / self.nobs
        self.llf = -0.5 * self.nobs * (np.log(2 * np.pi) + np.log(sigma2_mle) + 1)

        # AIC/BIC (using log-likelihood)
        self.aic = -2 * self.llf + 2 * self.X.shape[1]
        self.bic = -2 * self.llf + np.log(self.nobs) * self.X.shape[1]

        return self

    def summary(self):
        print("                            OLS Regression Results                            ")
        print("==============================================================================")
        print(f"{'Dep. Variable:':<25}{self.y.name:>25}")
        print(f"{'Model:':<25}{'OLS':>25}")
        print(f"{'Method:':<25}{'Least Squares':>25}")
        print(f"{'Date:':<25}{'...':>25}")
        print(f"{'Time:':<25}{'...':>25}")
        print(f"{'Log-Likelihood:':<25}{self.llf:>25.2f}")
        print(f"{'No. Observations:':<25}{self.nobs:>25}")
        print(f"{'Df Residuals:':<25}{self.nobs - self.X.shape[1]:>25}")
        print(f"{'Df Model:':<25}{self.X.shape[1]-1:>25}")
        print(f"{'Covariance Type:':<25}{'nonrobust':>25}")
        print("==============================================================================")
        print(f"{'':<15}{'coef':<10}{'std err':<10}{'t':<10}{'P>|t|':<10}{'[0.025':<10}{'0.975]':<10}")
        print("-------------------------------------------------------------------------------")
        for i, (coef, se, t, p) in enumerate(zip(self.coef, self.se, self.t_values, self.p_values)):
            name = self.X_names[i]
            ci_low = coef - 1.96 * se
            ci_high = coef + 1.96 * se
            print(f"{name:<15}{coef:<10.4f}{se:<10.3f}{t:<10.3f}{p:<10.3f}"
                  f"{ci_low:<10.3f}{ci_high:<10.3f}")
        print("==============================================================================")
        print(f"{'R-squared:':<25}{self.r_squared:.3f}")
        print(f"{'Adj. R-squared:':<25}{self.adj_r_squared:.3f}")
        print(f"{'F-statistic:':<25}{self.f_stat:.2f}")
        print(f"{'Prob (F-statistic):':<25}{self.f_pvalue:.2e}")
        print(f"{'AIC:':<25}{self.aic:.1f}")
        print(f"{'BIC:':<25}{self.bic:.1f}")

In [11]:
# CAPM Implementation
def calculate_capm(asset_ticker, market_ticker, risk_free_rate_annual=0.02, period='5y'):
    # Download data
    asset = yf.Ticker(asset_ticker)
    market = yf.Ticker(market_ticker)
    asset_prices = asset.history(period=period)['Close']
    market_prices = market.history(period=period)['Close']

    # Calculate monthly returns
    returns = pd.DataFrame({
        'Asset': asset_prices.resample('M').last().pct_change().dropna(),
        'Market': market_prices.resample('M').last().pct_change().dropna()
    }).dropna()

    # Calculate excess returns
    risk_free_monthly = risk_free_rate_annual / 12
    returns['Excess Asset'] = returns['Asset'] - risk_free_monthly
    returns['Excess Market'] = returns['Market'] - risk_free_monthly

    # Fit custom OLS model
    model = OLS(returns['Excess Asset'], returns['Excess Market'])
    results = model.fit()

    # Calculate CAPM components
    beta = results.coef[1]
    alpha = results.coef[0]
    expected_market_return = returns['Market'].mean() * 12
    expected_asset_return = (risk_free_rate_annual+beta * (expected_market_return - risk_free_rate_annual))

    # Print results
    results.summary()
    print("\nCAPM Expected Return:")
    print(f"  Beta (Systematic Risk): {beta:.4f}")
    print(f"  Alpha (Jensen's Alpha): {alpha:.4f}")
    print(f"  Expected Annual Return: {expected_asset_return:.2%}")

# Example usage
calculate_capm('AAPL', '^GSPC')

                            OLS Regression Results                            
Dep. Variable:                        Excess Asset
Model:                                         OLS
Method:                              Least Squares
Date:                                          ...
Time:                                          ...
Log-Likelihood:                              89.24
No. Observations:                               60
Df Residuals:                                   58
Df Model:                                        1
Covariance Type:                         nonrobust
               coef      std err   t         P>|t|     [0.025    0.975]    
-------------------------------------------------------------------------------
const          0.0067    0.007     0.916     0.363     -0.008    0.021     
Excess Market  1.2858    0.155     8.314     0.000     0.983     1.589     
R-squared:               0.544
Adj. R-squared:          0.536
F-statistic:             69.12
Prob (F-st

<ipython-input-11-c431cb2960f5>:11: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  'Asset': asset_prices.resample('M').last().pct_change().dropna(),
<ipython-input-11-c431cb2960f5>:12: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  'Market': market_prices.resample('M').last().pct_change().dropna()
